<a href="https://colab.research.google.com/github/mahmoodalzahawi/CNN-intro-code/blob/main/CNN_beta_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
# convert MNISt image files into tensor of 4-dimensions (of images, height, width, color channel)
transform = transforms.ToTensor()

In [32]:
#train
train_data=datasets.MNIST(root='/cnn_data',train=True,download=True,transform=transform)

In [33]:
#test
test_data=datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)


In [34]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [35]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [36]:
#create a small batch of images let's say 10
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
test_loader=DataLoader(test_data,batch_size=10,shuffle=False)

In [37]:
#define our cnn model
#decribe convo layer and what's it doing (2 covo layer)
conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1)
conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1)

In [38]:
#grab 1 MNIST record
for i,(X_train,y_train) in enumerate(train_data):
    break

In [39]:
X_train.shape

torch.Size([1, 28, 28])

In [40]:
x= X_train.view(1,1,28,28)

In [41]:
#perform our first convolution
x=f.relu(conv1(x)) #rectified Linear Unit fro our activation function



In [42]:
#1 single image
#6 is the fiilters we asked for
#26 by 26 is the image
x.shape

torch.Size([1, 6, 26, 26])

In [43]:
#pass thru the pooling layer
x= f.max_pool2d(x,2,2) #kernal of 2 and stride of 2

In [44]:
#in the pooling process your lost the un necesserary data by 2 as we defined in the pooling layer
x.shape

torch.Size([1, 6, 13, 13])

In [45]:
#now for the conv2
x=f.relu(conv2(x))

In [47]:
x.shape

torch.Size([1, 16, 11, 11])

In [48]:
#pooling layer
x=f.max_pool2d(x,2,2)

In [49]:
x.shape #we can't invent data around a half of a data so it's 5 not 6

torch.Size([1, 16, 5, 5])

In [53]:
#model class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1)
        #fully connected layers is configured after getting the features out
        #the 400 is 5*5*16 which is the final result of the feature extraction phase
        self.fc1=nn.Linear(in_features=400,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=84)
        #it has t0 end at 10 because of the number of clases in the dataset is 10
        self.fc3=nn.Linear(in_features=84,out_features=10)

    def forward(self,x):
      #first conv
        x=f.relu(self.conv1(x))
        x=f.max_pool2d(x,2,2)
        #second conv
        x=f.relu(self.conv2(x))
        x=f.max_pool2d(x,2,2)
        #we neeed to flatten our data and it's -1 so that we can vary our batch size
        x=x.view(-1,400)
        #fully connected layers
        x=f.relu(self.fc1(x))
        x=f.relu(self.fc2(x))
        x=self.fc3(x)
        return f.log_softmax(x,dim=1)

In [54]:
#create aninstance of our model
torch.manual_seed(41)
model=CNN()
model

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [55]:
#loss function optimizer
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)#the smaller the learning rate the longer it will take